# Step 3: DPA on Firmware Implementation of AES

**SUMMARY:** *In the previous step, you saw how a single bit of information can be used to recover an entire byte of the AES key. Remember, this works due to the S-Box being present in the data flow that we are attacking.*

*Next, we'll see how to use power analysis instead of an actual bit value. With this technique, the goal is to separate the traces by a bit in the result of the SBox output (it doesn't matter which one): if that bit is 1, its group of traces should, on average, have higher power consumption during the SBox operation than the other set.*

*This is all based on the assumption we discussed in the slides and saw in earlier labs: there is some consistent relationship between the value of bits on the data bus and the power consumption in the device.*

**LEARNING OUTCOMES:**

* Using a power measurement to 'validate' a possible device model.
* Detecting the value of a single bit using power measurement.
* Breaking AES using the classic DPA attack.


In [4]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER = 'SS_VER_2_1'

In [5]:
%run "Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [6]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
make[1]: '.dep' is up to date.
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_C

In [7]:
cw.program_target(scope, prog, "hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 10851 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 10851 bytes


In [16]:
import time


key = input("Please Provide the desired Key. It will be truncated or expanded to be exactly 16 Characters:")
key =bytearray(key.encode('utf-8'))
while len(key) % 16 != 0:
    key.append(0)

TruncatedKey = key[:15]

Data = input("Please Provide the desired Plaintext. Any overflow will be decrypted as a seperate message. Any underflow will be expanded to be exactly 16 characters:")
Data =bytearray(Data.encode('utf-8'))
while len(Data) % 16 != 0:
    Data.append(0)

TruncatedKey = key[:15]

target.simpleserial_write('l', TruncatedKey)

for i in range(0, len(Data), 16):
    #This needs to be here for the program to run
    target.simpleserial_read('r', 16, timeout=500)
    
    text = Data[i:i+16]

    print("This is the provided text:")
    print(text)

    target.simpleserial_write('e', text)

    response = target.simpleserial_read('r', 16, timeout=500)

    print("This is the encrypted text:")
    print(response)

    print("Now for Decryption...")


    target.simpleserial_write('d', response)


    decryption = target.simpleserial_read('r', 16, timeout=500)
    DecryptedPlain = decryption.decode("utf-8") 
    print("Decrytped Data: ", decryption)
    print("Decrypted ASCII:", DecryptedPlain)

Please Provide the desired Key. It will be truncated or expanded to be exactly 16 Characters:PLEASEPLEASEPLEASEPLEASEPLEASEPLEASEPLEASEPLEASE
Please Provide the desired Plaintext. Any overflow will be decrypted as a seperate message. Any underflow will be expanded to be exactly 16 characters:PLEASEPLEASEPLEASEPLEASEPLEASEPLEASEPLEASEPLEASE
This is the provided text:
bytearray(b'PLEASEPLEASEPLEA')


(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: eë 


This is the encrypted text:
None
Now for Decryption...


TypeError: object of type 'NoneType' has no len()